In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

In [16]:
url = "https://listado.mercadolibre.cl/electronica-audio-video/televisores/nuevo/#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondici%C3%B3n%26applied_filter_order%3D6%26applied_value_id%3D2230284%26applied_value_name%3DNuevo%26applied_value_order%3D1%26applied_value_results%3D421%26is_custom%3Dfalse"
pages = []
pages.append(url)
for i in range(7):
    i = requests.get(url).text
    soup=BeautifulSoup(i,'html.parser')
    link = soup.find("div",{"class":"ui-search-pagination"})
    link = link.find("li",{"class":"andes-pagination__button andes-pagination__button--next"})
    i = link.find("a",{"class":"andes-pagination__link ui-search-link"}).get('href')
    pages.append(i)
    url = i

In [17]:
productlists = []
for page in pages:
        k = requests.get(page).text
        soup=BeautifulSoup(k,'html.parser')
        productlist = soup.find_all("li",{"class":"ui-search-layout__item"})
        productlists.append(productlist)

from typing import Iterable 
#from collections import Iterable                            # < py38


def flatten(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x
            
list(flatten(productlists))

In [18]:
items = [item for sublist in productlists for item in sublist]

In [19]:
len(items)

412

In [20]:
productlinks = []
for item in items:
    for product in item:
        main = product.find("div", {"class":"ui-search-item__group ui-search-item__group--title"})
        link = main.find("a",{"class":"ui-search-item__group__element ui-search-link"}).get('href')                 
        productlinks.append(link)

In [21]:
data=[]
page = 1
for link in productlinks:
    from datetime import datetime
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    
    tables = hun.find_all('tbody', class_='andes-table__body')
    def get_specs(tables):
        column_names = []
        dt = []
        for table in tables:
            for row in table.find_all('th'):
                name = row.get_text()
                column_names.append(name)

            for row in table.find_all('td'):
                name = row.get_text()
                dt.append(name)
        
        specs = {column_names[i]: dt[i] for i in range(len(column_names))}
        return specs
    
    try:
        currency = hun.find("span",{"class":"andes-money-amount__currency-symbol"}).text.replace('\n',"")
    except:
        currency = None
    
    try:
        original=hun.find("s",{"class":"andes-money-amount ui-pdp-price__part ui-pdp-price__original-value andes-money-amount--previous andes-money-amount--cents-superscript andes-money-amount--compact"})
        original_price = original.find("span", {"class":"andes-visually-hidden"}).text.replace('\n',"")
    except:
        original_price = None


    try:
        price = hun.find("span",{"class":"andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact"}).text.replace('\n',"")
    except:
        price = None
        
    try:
        discount_percentage = hun.find("span",{"class":"andes-money-amount__discount"}).text.replace('\n',"")
    except:
        discount = None
        
    try:
        sales = hun.find("span",{"class":"ui-pdp-subtitle"}).text.replace('\n',"")
    except:
        sales = None
    
    try:
        quantity_available = hun.find("span", {"class":"ui-pdp-buybox__quantity__available"}).text.replace('\n',"")
    except:
        quantity_available = None
        
    try:
        reviews = hun.find("span",{"class":"ui-pdp-review__amount"}).text.replace('\n',"")
    except:
        reviews = None
        
    try:
        description = hun.find_all("li", class_="ui-vpp-highlighted-specs__features-list-item ui-pdp-color--BLACK ui-pdp-size--XSMALL ui-pdp-family--REGULAR")
        
    except:
        description = None
    
    try:
        specifications = get_specs(tables)
    except:
        specifications = None

    try:
        name=hun.find("h1",{"class":"ui-pdp-title"}).text.replace('\n',"")
    except:
        name=None
    
    try:
        category = soup.find("div",{"class":"ui-search-breadcrumb"})
        category = category.find("h1",{"class":"ui-search-breadcrumb__title"}).text.replace('\n',"")
    except:
        category = None
    
    try:
        rating = hun.find("div", {"class":"ui-review-view__rating__summary"})
        rating = rating.find("p", {"class":"ui-review-view__rating__summary__average"}).text.replace('\n',"")
    except:
        rating = None
        
    try:
        country_code = "CL"
    except:
        country_code = None
    try:
        scrape_link = link
        
    except:
        scrape_link = None
    
    try:
        now = datetime.now()
        date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    except:
        date_scraped = None
    
    print(f'Products Scraped: {page}', end =" ")
    
    page = page +1
    
    

    tv = {"name":name,   "Country Code": country_code,"rating":rating, "category":category,"Currency":currency, "original_price":original_price, "price":price, "discount_percentage":discount_percentage, "sales":sales, "quantity_available":quantity_available, "reviews":reviews, "description":description, "specifications":specifications, "scrape_link":scrape_link, "date_scraped":date_scraped}

    data.append(tv)

Products Scraped: 1 Products Scraped: 2 Products Scraped: 3 Products Scraped: 4 Products Scraped: 5 Products Scraped: 6 Products Scraped: 7 Products Scraped: 8 Products Scraped: 9 Products Scraped: 10 Products Scraped: 11 Products Scraped: 12 Products Scraped: 13 Products Scraped: 14 Products Scraped: 15 Products Scraped: 16 Products Scraped: 17 Products Scraped: 18 Products Scraped: 19 Products Scraped: 20 Products Scraped: 21 Products Scraped: 22 Products Scraped: 23 Products Scraped: 24 Products Scraped: 25 Products Scraped: 26 Products Scraped: 27 Products Scraped: 28 Products Scraped: 29 Products Scraped: 30 Products Scraped: 31 Products Scraped: 32 Products Scraped: 33 Products Scraped: 34 Products Scraped: 35 Products Scraped: 36 Products Scraped: 37 Products Scraped: 38 Products Scraped: 39 Products Scraped: 40 Products Scraped: 41 Products Scraped: 42 Products Scraped: 43 Products Scraped: 44 Products Scraped: 45 Products Scraped: 46 Products Scraped: 47 Products Scraped: 48 P

In [43]:
df = pd.DataFrame.from_dict(data)
df.shape

(412, 15)

In [44]:
df.head()

,name,Country Code,rating,category,Currency,original_price,price,discount_percentage,sales,quantity_available,reviews,description,specifications,scrape_link,date_scraped
0,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",CL,4.7,Televisores Nuevo,$,Precio anterior: 179990 pesos,129990 pesos$129.990,27% OFF,Nuevo | 3076 vendidos,(190 disponibles),(187),"[[Su resolución es Full HD.], [Modo de sonido:...","{'Marca': 'Aiwa', 'Modelo': 'AW32B4SM', 'Volta...",https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,04/08/2022 19:17:16
1,Smart Tv Caixun 43 Fhd Android C43v1fa,CL,4.6,Televisores Nuevo,$,Precio anterior: 299990 pesos,149990 pesos$149.990,50% OFF,Nuevo | 2247 vendidos,(339 disponibles),(114),[],"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933661578...,04/08/2022 19:17:18
2,"Smart TV LG AI ThinQ 32LM637BPSB LED HD 32"" 10...",CL,4.8,Televisores Nuevo,$,None,239990 pesos$239.990,50% OFF,Nuevo | 1899 vendidos,(18 disponibles),(107),"[[Su resolución es HD.], [Tecnología HDR para ...","{'Marca': 'LG', 'Línea': 'AI ThinQ', 'Modelo':...",https://www.mercadolibre.cl/smart-tv-lg-ai-thi...,04/08/2022 19:17:21
3,Smart Tv Caixun 32 Hd Android C32v1ha,CL,4.6,Televisores Nuevo,$,Precio anterior: 189990 pesos,109990 pesos$109.990,42% OFF,Nuevo | 4123 vendidos,(821 disponibles),(208),[],"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933647828...,04/08/2022 19:17:23
4,"Smart TV TCL S6500-Series 42S6500 LED Full HD 42""",CL,4.6,Televisores Nuevo,$,None,259990 pesos$259.990,42% OFF,Nuevo | 2548 vendidos,(7 disponibles),(138),"[[Su resolución es Full HD.], [Tecnología HDR ...","{'Marca': 'TCL', 'Línea': 'S6500-Series', 'Mod...",https://www.mercadolibre.cl/smart-tv-tcl-s6500...,04/08/2022 19:17:25


In [38]:
##Convert to string in order to use replace
df['description'] = df['description'].astype('str') 
##remove html tags
df['description'] = df['description'].str.replace(r'<[^<>]*>', '', regex=True)
##remove square brackets
df['description'] =  df['description'].apply(lambda x: x.replace('[','').replace(']','')) 
df.head()

,name,Country Code,rating,category,Currency,original_price,price,discount_percentage,sales,quantity_available,reviews,description,specifications,scrape_link,date_scraped
0,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",CL,4.7,Televisores Nuevo,$,Precio anterior: 179990 pesos,129990 pesos$129.990,27% OFF,Nuevo | 3076 vendidos,(190 disponibles),(187),"Su resolución es Full HD., Modo de sonido: Ste...","{'Marca': 'Aiwa', 'Modelo': 'AW32B4SM', 'Volta...",https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,04/08/2022 19:17:16
1,Smart Tv Caixun 43 Fhd Android C43v1fa,CL,4.6,Televisores Nuevo,$,Precio anterior: 299990 pesos,149990 pesos$149.990,50% OFF,Nuevo | 2247 vendidos,(339 disponibles),(114),,"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933661578...,04/08/2022 19:17:18
2,"Smart TV LG AI ThinQ 32LM637BPSB LED HD 32"" 10...",CL,4.8,Televisores Nuevo,$,None,239990 pesos$239.990,50% OFF,Nuevo | 1899 vendidos,(18 disponibles),(107),"Su resolución es HD., Tecnología HDR para una ...","{'Marca': 'LG', 'Línea': 'AI ThinQ', 'Modelo':...",https://www.mercadolibre.cl/smart-tv-lg-ai-thi...,04/08/2022 19:17:21
3,Smart Tv Caixun 32 Hd Android C32v1ha,CL,4.6,Televisores Nuevo,$,Precio anterior: 189990 pesos,109990 pesos$109.990,42% OFF,Nuevo | 4123 vendidos,(821 disponibles),(208),,"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933647828...,04/08/2022 19:17:23
4,"Smart TV TCL S6500-Series 42S6500 LED Full HD 42""",CL,4.6,Televisores Nuevo,$,None,259990 pesos$259.990,42% OFF,Nuevo | 2548 vendidos,(7 disponibles),(138),"Su resolución es Full HD., Tecnología HDR para...","{'Marca': 'TCL', 'Línea': 'S6500-Series', 'Mod...",https://www.mercadolibre.cl/smart-tv-tcl-s6500...,04/08/2022 19:17:25


In [39]:
##Convert to string in order to use replace
df['reviews'] = df['reviews'].astype('str') 
df['reviews'] = df['reviews'].str.replace(r'[()]',"")

C:\Users\user\AppData\Local\Temp\ipykernel_4216\994336770.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews'] = df['reviews'].str.replace(r'[()]',"")


In [40]:
df['reviews'] = pd.to_numeric(df['reviews'], errors='ignore')
df.head()

,name,Country Code,rating,category,Currency,original_price,price,discount_percentage,sales,quantity_available,reviews,description,specifications,scrape_link,date_scraped
0,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",CL,4.7,Televisores Nuevo,$,Precio anterior: 179990 pesos,129990 pesos$129.990,27% OFF,Nuevo | 3076 vendidos,(190 disponibles),187,"Su resolución es Full HD., Modo de sonido: Ste...","{'Marca': 'Aiwa', 'Modelo': 'AW32B4SM', 'Volta...",https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,04/08/2022 19:17:16
1,Smart Tv Caixun 43 Fhd Android C43v1fa,CL,4.6,Televisores Nuevo,$,Precio anterior: 299990 pesos,149990 pesos$149.990,50% OFF,Nuevo | 2247 vendidos,(339 disponibles),114,,"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933661578...,04/08/2022 19:17:18
2,"Smart TV LG AI ThinQ 32LM637BPSB LED HD 32"" 10...",CL,4.8,Televisores Nuevo,$,None,239990 pesos$239.990,50% OFF,Nuevo | 1899 vendidos,(18 disponibles),107,"Su resolución es HD., Tecnología HDR para una ...","{'Marca': 'LG', 'Línea': 'AI ThinQ', 'Modelo':...",https://www.mercadolibre.cl/smart-tv-lg-ai-thi...,04/08/2022 19:17:21
3,Smart Tv Caixun 32 Hd Android C32v1ha,CL,4.6,Televisores Nuevo,$,Precio anterior: 189990 pesos,109990 pesos$109.990,42% OFF,Nuevo | 4123 vendidos,(821 disponibles),208,,"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933647828...,04/08/2022 19:17:23
4,"Smart TV TCL S6500-Series 42S6500 LED Full HD 42""",CL,4.6,Televisores Nuevo,$,None,259990 pesos$259.990,42% OFF,Nuevo | 2548 vendidos,(7 disponibles),138,"Su resolución es Full HD., Tecnología HDR para...","{'Marca': 'TCL', 'Línea': 'S6500-Series', 'Mod...",https://www.mercadolibre.cl/smart-tv-tcl-s6500...,04/08/2022 19:17:25


In [49]:
df['Currency'] = df['price'].str[7:12]

In [50]:
df['state'] = df['sales'].str[:6]
df['state'] = df['state'].astype('str') 

In [52]:
##Feature Engineering
df['discount_percentage'] = df['discount_percentage'].str.extract('(\d+)', expand=False)
df['sales'] = df['sales'].str.extract('(\d+)', expand=False)
df['quantity_available'] = df['quantity_available'].str.extract('(\d+)', expand=False)
df['price'] = df['price'].str.extract('(\d+)', expand=False)
df['original_price'] = df['original_price'].str.extract('(\d+)', expand=False)
df.head()

,name,Country Code,rating,category,Currency,original_price,price,discount_percentage,sales,quantity_available,reviews,description,specifications,scrape_link,date_scraped,state
0,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",CL,4.7,Televisores Nuevo,pesos,179990,129990,27,3076,190,(187),"[[Su resolución es Full HD.], [Modo de sonido:...","{'Marca': 'Aiwa', 'Modelo': 'AW32B4SM', 'Volta...",https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,04/08/2022 19:17:16,Nuevo
1,Smart Tv Caixun 43 Fhd Android C43v1fa,CL,4.6,Televisores Nuevo,pesos,299990,149990,50,2247,339,(114),[],"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933661578...,04/08/2022 19:17:18,Nuevo
2,"Smart TV LG AI ThinQ 32LM637BPSB LED HD 32"" 10...",CL,4.8,Televisores Nuevo,pesos,None,239990,50,1899,18,(107),"[[Su resolución es HD.], [Tecnología HDR para ...","{'Marca': 'LG', 'Línea': 'AI ThinQ', 'Modelo':...",https://www.mercadolibre.cl/smart-tv-lg-ai-thi...,04/08/2022 19:17:21,Nuevo
3,Smart Tv Caixun 32 Hd Android C32v1ha,CL,4.6,Televisores Nuevo,pesos,189990,109990,42,4123,821,(208),[],"{'Marca': 'Caixun', 'Línea': 'Serie V', 'Model...",https://articulo.mercadolibre.cl/MLC-933647828...,04/08/2022 19:17:23,Nuevo
4,"Smart TV TCL S6500-Series 42S6500 LED Full HD 42""",CL,4.6,Televisores Nuevo,pesos,None,259990,42,2548,7,(138),"[[Su resolución es Full HD.], [Tecnología HDR ...","{'Marca': 'TCL', 'Línea': 'S6500-Series', 'Mod...",https://www.mercadolibre.cl/smart-tv-tcl-s6500...,04/08/2022 19:17:25,Nuevo


In [53]:
df['quantity_available'] = df['quantity_available'].fillna(1) #Where quantity is empty replace with 1


In [54]:
spec_df = df['specifications'].apply(pd.Series)
df = pd.concat([df, spec_df.reindex(df.index)], axis=1)
# Remove specifications column
#df.drop(['specifications'], axis=1, inplace=True)
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_4216\2626193866.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  spec_df = df['specifications'].apply(pd.Series)
C:\Users\user\AppData\Local\Temp\ipykernel_4216\2626193866.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  spec_df = df['specifications'].apply(pd.Series)
C:\Users\user\AppData\Local\Temp\ipykernel_4216\2626193866.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  spec_df = df['specifications'].apply(pd.Series)
C:\Users\user\AppData\Local\Temp\ipykernel_4216\2626193866.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version

,name,Country Code,rating,category,Currency,original_price,price,discount_percentage,sales,quantity_available,...,Ancho,Países con control mágico incluido,Con cables ocultos,Profundidad,Altura,Ancho x Profundidad,Ancho x Altura,Número de registro/certificación INMETRO,Organismo de certificación (OCP),Número de certificado de seguridad eléctrica
0,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",CL,4.7,Televisores Nuevo,pesos,179990,129990,27,3076,190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Smart Tv Caixun 43 Fhd Android C43v1fa,CL,4.6,Televisores Nuevo,pesos,299990,149990,50,2247,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Smart TV LG AI ThinQ 32LM637BPSB LED HD 32"" 10...",CL,4.8,Televisores Nuevo,pesos,None,239990,50,1899,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Smart Tv Caixun 32 Hd Android C32v1ha,CL,4.6,Televisores Nuevo,pesos,189990,109990,42,4123,821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Smart TV TCL S6500-Series 42S6500 LED Full HD 42""",CL,4.6,Televisores Nuevo,pesos,None,259990,42,2548,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_l  = pd.concat([df[['Country Code', 'name', 'quantity_available', 'state', 'Currency', 'original_price', 'price', 'sales', 'category', 'date_scraped', 'scrape_link']], spec_df.reindex(df.index)], axis=1)
spec_df.columns

Index(['Marca', 'Modelo', 'Voltaje', 'Frecuencia', 'Es smart',
       'Sistema operativo', 'Con apagado automático',
       'Ancho x Profundidad x Altura', 'Ancho con soporte',
       'Profundidad con soporte', 'Altura con soporte',
       'Cantidad de parlantes', 'Potencia máxima de los parlantes',
       'Modos de sonido', 'Apps integradas', 'Tipo de pantalla',
       'Tamaño de la pantalla', 'Tipo de resolución', 'Resolución máxima',
       'Relación de aspecto', 'Frecuencia de actualización de la pantalla',
       'Con USB', 'Con HDMI', 'Cantidad de puertos HDMI',
       'Cantidad de puertos USB', 'Con Wi-Fi', 'Con ethernet', 'Con VGA',
       'Con entrada S/PDIF', 'Con entrada por video componentes',
       'Con entrada de antena RF', 'Línea', 'Modelo alfanumérico', 'Color',
       'Es 3D', 'Es curvo', 'Es portátil',
       'Cantidad de núcleos del procesador', 'Peso',
       'Con función screen share', 'Con HDR', 'Relación de contraste',
       'Con Bluetooth', 'Peso con soporte'

In [56]:
df_long = df_l.set_index(['Country Code', 'Marca', 'Modelo', 'Modelo alfanumérico', 'category', 'name', 'quantity_available', 'state', 'Currency', 'original_price','price','sales','date_scraped', 'scrape_link']).stack().reset_index()
df_long.head()

,Country Code,Marca,Modelo,Modelo alfanumérico,category,name,quantity_available,state,Currency,original_price,price,sales,date_scraped,scrape_link,level_14,0
0,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Voltaje,100V/240V
1,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Frecuencia,50 Hz x 60 Hz
2,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Es smart,Sí
3,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Sistema operativo,Linux
4,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Con apagado automático,Sí


In [57]:
df_long

,Country Code,Marca,Modelo,Modelo alfanumérico,category,name,quantity_available,state,Currency,original_price,price,sales,date_scraped,scrape_link,level_14,0
0,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Voltaje,100V/240V
1,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Frecuencia,50 Hz x 60 Hz
2,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Es smart,Sí
3,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Sistema operativo,Linux
4,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Con apagado automático,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7572,CL,Generic,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Televisores Nuevo,Tv Stick Lite Full Hd 8gb Negro Y 1 Gb Ram,20,Nuevo,esos$,NaN,29439,NaN,04/08/2022 19:30:50,https://articulo.mercadolibre.cl/MLC-103601685...,Es portátil,Sí
7573,CL,Generic,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Televisores Nuevo,Tv Stick Lite Full Hd 8gb Negro Y 1 Gb Ram,20,Nuevo,esos$,NaN,29439,NaN,04/08/2022 19:30:50,https://articulo.mercadolibre.cl/MLC-103601685...,Con HDR,Sí
7574,CL,Generic,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Televisores Nuevo,Tv Stick Lite Full Hd 8gb Negro Y 1 Gb Ram,20,Nuevo,esos$,NaN,29439,NaN,04/08/2022 19:30:50,https://articulo.mercadolibre.cl/MLC-103601685...,Número de registro/certificación INMETRO,1
7575,CL,Generic,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Amazon Fire TV Stick Lite Full HD 8GB Black wi...,Televisores Nuevo,Tv Stick Lite Full Hd 8gb Negro Y 1 Gb Ram,20,Nuevo,esos$,NaN,29439,NaN,04/08/2022 19:30:50,https://articulo.mercadolibre.cl/MLC-103601685...,Organismo de certificación (OCP),Qetkra S.R.L.


In [58]:
df_long.rename(columns = {'level_14':'Specification Name', 0:'Specification Value'}, inplace=True)


In [59]:
df_long.head()

,Country Code,Marca,Modelo,Modelo alfanumérico,category,name,quantity_available,state,Currency,original_price,price,sales,date_scraped,scrape_link,Specification Name,Specification Value
0,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Voltaje,100V/240V
1,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Frecuencia,50 Hz x 60 Hz
2,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Es smart,Sí
3,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Sistema operativo,Linux
4,CL,Aiwa,AW32B4SM,NaN,Televisores Nuevo,"Smart TV Aiwa AW32B4SM LED Full HD 32"" 100V/240V",190,Nuevo,pesos,179990,129990,3076,04/08/2022 19:17:16,https://www.mercadolibre.cl/smart-tv-aiwa-aw32...,Con apagado automático,Sí


In [10]:
url = "https://www.mercadolibre.cl/smart-tv-aoc-32s519578g-led-hd-32/p/MLC15581929?pdp_filters=ITEM_CONDITION:2230284%7Ccategory:MLC1002#searchVariation=MLC15581929&position=43&search_layout=stack&type=product&tracking_id=a1308394-f0fa-4977-bb68-137ab33f9f41"
f = requests.get(url,headers=headers).text
hun=BeautifulSoup(f,'html.parser')
price_div = hun.find('ul', class_='ui-pdp-products__list')



try:
    price = price_div.find("span",{"class":"andes-money-amount ui-pdp-price__part andes-money-amount--cents-comma andes-money-amount--compact"}).text.replace('\n',"")
except:
    price = None
    
print(price)


148990 pesos$148.990


In [11]:
price[:6] #price

'148990'

In [14]:
price[6:12] #currency

' pesos'

In [7]:
price

In [60]:
df_long.to_csv("tv_data_long.CSV")

In [61]:
df.to_csv('tv_data.csv')